In [29]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
import pandas as pd


In [2]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [3]:
# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

In [4]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0985e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9951e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2531e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0639e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6242e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4718e

In [5]:
# Load the user pref full cleaned
df_sentences = pd.read_csv(r'../../../user_preferences_full_cleaned.csv',sep=';')

df_sentences.head()

,userId,PeliculasFavoritas,TitulosPeliculas,SeriesFavoritas,TitulosSeries,GenerosFavoritos,DetallesPeliculas,DetallesSeries,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,3169503e-8011-707f-a637-23e5f9a37d5d,653346;693134;996154,El reino del planeta de los simios;Dune: Parte...,121,Doctor Who,"['Accion', 'Aventura', 'Animacion', 'Comedia',...","[{""Titulo"": ""El reino del planeta de los simio...","[{""Titulo"": ""Doctor Who"", ""Sinopsis"": ""Doctor ...",NaN,NaN,NaN,NaN,NaN
1,"vive entre la luz y la oscuridad, llena de per...",no pueden regresar a casa porque la ley los p...,los mutantes. Dotados de extranos y variados ...,aparecer en su concurso televisivo preferido....,"de lo contrario, su existencia no seria posib...","[{""Titulo"": ""Juego de tronos"", ""Sinopsis"": ""En...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,".org/t/p/w500/7uintpyNlSF32hj5Bua85VuAp2R.jpg""...",los mutantes. Dotados de extranos y variados ...,"tambien Peter se transforma, sacando el lado ...",es una especie de pildora inteligente gracias...,Manny y Ellie esperan el nacimiento de su min...,"Sid, el perezoso, se mete en un buen lio al f...",pero durante la noche debe esconderse de ello...,todos los dias envia mensajes por radio con l...,un imponente Capitolio ejerce un control rigu...,"pero para ella, que ya ha visto la muerte de ...",NaN,NaN,NaN
3,"a a su segundo ano a Hogwarts, pero descubre q...","tambien Peter se transforma, sacando el lado ...","en cambio, la autodestruccion es lo unico que...",los mutantes. Dotados de extranos y variados ...,"[{""Titulo"": ""Juego de tronos"", ""Sinopsis"": ""En...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"o"""": 7.02}, {""""Titulo"""": """"Frozen: El reino de...",un imponente Capitolio ejerce un control rigu...,"pero para ella, que ya ha visto la muerte de ...","nunca se conformaban con lo que tenian."""", """"...","sin embargo, debido a su precaria salud, fue ...",salvar la unidad del grupo y la amistad que l...,"tambien Peter se transforma, sacando el lado ...","[{""Titulo"": ""Juego de tronos"", ""Sinopsis"": ""En...",NaN,NaN,NaN,NaN,NaN


In [6]:
# Filter rows where the 'userId' column has 36 characters
df_sentences = df_sentences[df_sentences['userId'].str.len()==36]
df_sentences.head()

,userId,PeliculasFavoritas,TitulosPeliculas,SeriesFavoritas,TitulosSeries,GenerosFavoritos,DetallesPeliculas,DetallesSeries,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,3169503e-8011-707f-a637-23e5f9a37d5d,653346;693134;996154,El reino del planeta de los simios;Dune: Parte...,121,Doctor Who,"['Accion', 'Aventura', 'Animacion', 'Comedia',...","[{""Titulo"": ""El reino del planeta de los simio...","[{""Titulo"": ""Doctor Who"", ""Sinopsis"": ""Doctor ...",NaN,NaN,NaN,NaN,NaN
26,e129f0be-7021-70f6-8c43-24806020bacd,105;862;2062;424694;313369,Regreso al futuro;Toy Story;Ratatouille;Bohemi...,100883;31356;31251;97186;72637,"Yo nunca;Big Time Rush;Victorious;Con amor, Vi...","['Comedia', 'Familia', 'Romance', 'Musica', 'P...","[{""Titulo"": ""Regreso al futuro"", ""Sinopsis"": ""...","[{""Titulo"": ""Yo nunca"", ""Sinopsis"": ""Tras un a...",NaN,NaN,NaN,NaN,NaN
27,c109908e-60d1-7078-8255-cb99805d438a,274870;157336;1265493;27205;286217,Passengers;Interstellar;Hispanoamerica: canto ...,106379;74851;63247,Fallout;Mas alla de Stranger Things;Westworld,"['Ciencia ficcion', 'Suspense', 'Misterio', 'F...","[{""Titulo"": ""Passengers"", ""Sinopsis"": ""Una nav...","[{""Titulo"": ""Fallout"", ""Sinopsis"": ""Basado en ...",NaN,NaN,NaN,NaN,NaN
28,d14910ee-f041-7052-1117-9987d4514d42,155;19995;24428;299536;118340,El caballero oscuro;Avatar;Los Vengadores;Veng...,119051;12637;1434;62688;16286;33761,Miercoles;Rebelde;Padre de familia;Supergirl;Y...,"['Accion', 'Fantasia', 'Familia', 'Comedia', '...","[{""Titulo"": ""El caballero oscuro"", ""Sinopsis"":...","[{""Titulo"": ""Miercoles"", ""Sinopsis"": ""Miercole...",NaN,NaN,NaN,NaN,NaN
29,41a9407e-3091-70bf-390e-efbfd972295d,155;24428;293660;299536;118340,El caballero oscuro;Los Vengadores;Deadpool;Ve...,119051;1412;46331;79242;63926,Miercoles;Arrow;La cupula;Las escalofriantes a...,"['Accion', 'Animacion', 'Comedia', 'Familia', ...","[{""Titulo"": ""El caballero oscuro"", ""Sinopsis"":...","[{""Titulo"": ""Miercoles"", ""Sinopsis"": ""Miercole...",NaN,NaN,NaN,NaN,NaN


In [7]:
df_sentences.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4213 entries, 0 to 4237
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userId              4213 non-null   object
 1   PeliculasFavoritas  4213 non-null   object
 2   TitulosPeliculas    4213 non-null   object
 3   SeriesFavoritas     4213 non-null   object
 4   TitulosSeries       4213 non-null   object
 5   GenerosFavoritos    4213 non-null   object
 6   DetallesPeliculas   4213 non-null   object
 7   DetallesSeries      4171 non-null   object
 8   Unnamed: 8          0 non-null      object
 9   Unnamed: 9          0 non-null      object
 10  Unnamed: 10         0 non-null      object
 11  Unnamed: 11         0 non-null      object
 12  Unnamed: 12         0 non-null      object
dtypes: object(13)
memory usage: 460.8+ KB


In [8]:
# Drop unnecessary columns, specifically those with names starting with 'Unnamed'
columns_to_drop = [col for col in df_sentences if col.startswith('Unnamed')]
df_sentences = df_sentences.drop(columns_to_drop, axis=1)
df_sentences.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4213 entries, 0 to 4237
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userId              4213 non-null   object
 1   PeliculasFavoritas  4213 non-null   object
 2   TitulosPeliculas    4213 non-null   object
 3   SeriesFavoritas     4213 non-null   object
 4   TitulosSeries       4213 non-null   object
 5   GenerosFavoritos    4213 non-null   object
 6   DetallesPeliculas   4213 non-null   object
 7   DetallesSeries      4171 non-null   object
dtypes: object(8)
memory usage: 296.2+ KB


In [24]:

pd.set_option('display.max_colwidth', 150)

In [10]:
# Show the type of the first value in the 'DetallesPeliculas' column
print(type(df_sentences['DetallesPeliculas'].iloc[0]))

# Display the first three rows of the 'DetallesPeliculas' column for inspection
print(df_sentences['DetallesPeliculas'].head(3))


<class 'str'>
0     [{"Titulo": "El reino del planeta de los simios", "Sinopsis": "Ambientada varias generaciones en el futuro tras el reinado de Cesar, en la que los...
26    [{"Titulo": "Regreso al futuro", "Sinopsis": "El adolescente Marty McFly es amigo de Doc, un cientifico al que todos toman por loco. Cuando Doc cr...
27    [{"Titulo": "Passengers", "Sinopsis": "Una nave espacial, que viaja a un planeta lejano transportando a miles de personas, tiene una averia en una...
Name: DetallesPeliculas, dtype: object


In [11]:
import ast

# Function to safely convert JSON-like strings to Python objects
def safe_literal_eval(x):
    try:
        # Attempt to parse the string as a Python object (e.g., list of dictionaries)
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        # If an error occurs, None
        return None

# Apply the function to the 'DetallesPeliculas' column
new_sentences = df_sentences['DetallesPeliculas'].apply(safe_literal_eval)

# Filtramos las filas donde la conversión fue exitosa (no es None)
new_sentences = new_sentences.dropna()

# Filter out rows where the conversion was unsuccessful (results in None)
sinopsis_list = [
    pelicula.get('Sinopsis') 
    for lista in new_sentences  
    for pelicula in lista  
    if isinstance(pelicula, dict) and 'Sinopsis' in pelicula  
]


new_sentences = sinopsis_list


['Ambientada varias generaciones en el futuro tras el reinado de Cesar, en la que los simios son la especie dominante que vive en armonia y los humanos se han visto reducidos a vivir en la sombra. Mientras un nuevo y tiranico lider simio construye su imperio, un joven simio emprende un angustioso viaje que le llevara a cuestionarse todo lo que sabe sobre el pasado y a tomar decisiones que definiran el futuro de simios y humanos por igual.', 'Sigue el viaje mitico de Paul Atreides mientras se une a Chani y los Fremen en una guerra de venganza contra los conspiradores que destruyeron a su familia. Al enfrentarse a una eleccion entre el amor de su vida y el destino del universo conocido, Paul se esfuerza por evitar un futuro terrible que solo el puede prever.', 'Un exagente de las fuerzas especiales libra una batalla de un solo hombre contra todos por las calles de Amsterdam. Todo con un unico objetivo: rescatar a la hija de su difunto amigo secuestrada por el sindicato del crimen local.'

In [22]:
sentences_try = new_sentences[:10]
sentences_try

['Ambientada varias generaciones en el futuro tras el reinado de Cesar, en la que los simios son la especie dominante que vive en armonia y los humanos se han visto reducidos a vivir en la sombra. Mientras un nuevo y tiranico lider simio construye su imperio, un joven simio emprende un angustioso viaje que le llevara a cuestionarse todo lo que sabe sobre el pasado y a tomar decisiones que definiran el futuro de simios y humanos por igual.',
 'Sigue el viaje mitico de Paul Atreides mientras se une a Chani y los Fremen en una guerra de venganza contra los conspiradores que destruyeron a su familia. Al enfrentarse a una eleccion entre el amor de su vida y el destino del universo conocido, Paul se esfuerza por evitar un futuro terrible que solo el puede prever.',
 'Un exagente de las fuerzas especiales libra una batalla de un solo hombre contra todos por las calles de Amsterdam. Todo con un unico objetivo: rescatar a la hija de su difunto amigo secuestrada por el sindicato del crimen local

In [25]:
pd.set_option('display.max_rows', 100)  												# Número máximo de filas a mostrar
pd.set_option('display.max_columns', 50)  												# Número máximo de columnas a mostrar

In [26]:
# Tokenize sentences second atempt
encoded_input_2 = tokenizer(sentences_try, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input_2)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input_2['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 0.0534,  0.0257, -0.0023,  ...,  0.0608,  0.0064, -0.0092],
        [-0.0698,  0.0487,  0.0031,  ...,  0.0452,  0.0245, -0.1092],
        [ 0.0350,  0.1146, -0.0110,  ...,  0.0786, -0.0124,  0.0014],
        ...,
        [ 0.0137, -0.0419, -0.0331,  ...,  0.0927,  0.0033, -0.0529],
        [ 0.0506,  0.0691,  0.0499,  ...,  0.0507, -0.0477, -0.0372],
        [-0.0058, -0.0189,  0.0006,  ...,  0.0279, -0.0263, -0.0038]])


In [30]:
# Select a user's embedding (first tensor as an example)
user_embedding = sentence_embeddings[0]

# Calculate the cosine similarity between the user's embedding and all content embeddings
content_similarities = cosine_similarity(user_embedding.unsqueeze(0), sentence_embeddings)

# Sort the indices by similarity score in descending order
most_similar_indices = content_similarities.argsort(descending=True)

# Display the top indices and their similarity scores
print("Índices más similares:", most_similar_indices[:5])
print("Similitudes más altas:", content_similarities[most_similar_indices[:5]])


Índices más similares: tensor([0, 9, 2, 4, 8])
Similitudes más altas: tensor([1.0000, 0.4874, 0.4267, 0.4264, 0.3883])


In [2]:
# will continue from here; now we know it works, start working on comparing metrics
